<a href="https://colab.research.google.com/github/Laura853/CP-IA/blob/main/extra_linear_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Introdução

O projeto tem como objetivo desenvolver e otimizar um modelo de Regressão Linear Multivariada para previsão do preço de venda de imóveis, utilizando um conjunto de dados contendo diversas características das propriedades (como tamanho, número de quartos, localização, entre outros).

A meta principal definida para o projeto é atingir um erro quadrático médio (RMSE) inferior a 130.000, demonstrando a capacidade preditiva e a eficiência do modelo em estimar preços próximos aos valores reais de mercado.

Inicialmente, foram implementados modelos de regressão linear simples e um pipeline com seleção de features. Contudo, ambos apresentaram desempenho abaixo da meta esperada.
Assim, neste notebook, foram aplicadas técnicas de otimização mantendo a natureza linear do modelo, incluindo:

Padronização dos dados (StandardScaler), para equilibrar escalas e melhorar a convergência;

Remoção de outliers via método interquartil (IQR), reduzindo distorções nos parâmetros;

Geração de termos polinomiais de grau 2 (PolynomialFeatures), permitindo capturar relações não lineares entre variáveis explicativas;

Seleção de variáveis (SelectKBest) com base na correlação estatística (f_regression);

Busca de hiperparâmetros (GridSearchCV), para definir automaticamente o melhor número de variáveis e o grau polinomial ideal.

Essas melhorias visam aumentar a precisão preditiva do modelo e reduzir o RMSE abaixo do limite estipulado, sem alterar a estrutura conceitual do projeto ou recorrer a algoritmos não lineares.

In [12]:
# 1) Encontrar e carregar o dataset automaticamente
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
import warnings

In [13]:
# Preparar os dados
from sklearn.datasets import fetch_openml

dados = fetch_openml(data_id=42731)

dados

{'data':        bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
 0             3       1.00         1180      5650     1.0           0     0   
 1             3       2.25         2570      7242     2.0           0     0   
 2             2       1.00          770     10000     1.0           0     0   
 3             4       3.00         1960      5000     1.0           0     0   
 4             3       2.00         1680      8080     1.0           0     0   
 ...         ...        ...          ...       ...     ...         ...   ...   
 21608         3       2.50         1530      1131     3.0           0     0   
 21609         4       2.50         2310      5813     2.0           0     0   
 21610         2       0.75         1020      1350     2.0           0     0   
 21611         3       2.50         1600      2388     2.0           0     0   
 21612         2       0.75         1020      1076     2.0           0     0   
 
        condition  grade  sqft

In [14]:
# 2) Inspeção rápida e definição de variável resposta
import numpy as np
df = dados.frame
print('Shape:', df.shape)
print('\nColunas:\n', df.columns.tolist())

# Procurar colunas candidatas para o target
candidates = [c for c in df.columns if any(x in c.lower() for x in ['price','preco','valor','sale'])]
if candidates:
    target_col = candidates[0]
else:
    # fallback: escolher a última coluna numérica
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if not num_cols:
        raise ValueError('Nenhuma coluna numérica encontrada para ser target.')
    target_col = num_cols[-1]

print('Coluna alvo escolhida:', target_col)
y = df[target_col].copy()
X = df.drop(columns=[target_col])

Shape: (21613, 22)

Colunas:
 ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'date_year', 'date_month', 'date_day']
Coluna alvo escolhida: price


In [15]:
# 3) Limpeza básica: remover NA, codificar categóricas com get_dummies, e padronizar tipos
df_orig = df.copy()

# Drop linhas com NA na target
mask = y.notna()
X = X[mask]
y = y[mask]

X = pd.get_dummies(X, drop_first=True)

print('Depois de get_dummies, shape X:', X.shape)

Depois de get_dummies, shape X: (21613, 89)


In [16]:
# 4) Remoção de outliers (IQR) nas features numéricas e no target
from scipy import stats
import numpy as np

def remove_outliers_iqr(X, y, factor=1.5):
    num_idx = X.select_dtypes(include=[np.number]).columns
    Xn = X.copy()
    mask = np.ones(len(Xn), dtype=bool)
    # aplicando IQR por coluna numérica
    for col in num_idx:
        q1 = Xn[col].quantile(0.25)
        q3 = Xn[col].quantile(0.75)
        iqr = q3 - q1
        low = q1 - factor * iqr
        high = q3 + factor * iqr
        mask &= (Xn[col] >= low) & (Xn[col] <= high)
    # filtrando target por IQR
    q1 = y.quantile(0.25); q3 = y.quantile(0.75); iqr = q3 - q1
    low = q1 - factor * iqr; high = q3 + factor * iqr
    mask &= (y >= low) & (y <= high)
    return Xn[mask], y[mask]

X_clean, y_clean = remove_outliers_iqr(X, y, factor=1.5)
print('Antes:', X.shape, 'Depois:', X_clean.shape)

Antes: (21613, 89) Depois: (14934, 89)


In [17]:
# 5) Divisão treino/teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)
print('Treino:', X_train.shape, 'Teste:', X_test.shape)

Treino: (11947, 89) Teste: (2987, 89)


In [18]:
# 6) Pipeline: StandardScaler -> PolynomialFeatures -> SelectKBest -> LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)),
    ('select', SelectKBest(score_func=f_regression)),
    ('lr', LinearRegression())
])

# Grid para buscar melhor número de features após seleção e grau polinomial (1 ou 2)
param_grid = {
    'poly__degree': [1, 2],
    # experimentando alguns valores de k proporcional ao número de features
    'select__k': [10, 20, 30, 'all']
}

# scorer usa neg_mean_squared_error (GridSearch maximiza)
gscv = GridSearchCV(pipe, param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=2)
gscv.fit(X_train, y_train)

print('Melhor combo:', gscv.best_params_)
print('Melhor score (neg MSE):', gscv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Melhor combo: {'poly__degree': 2, 'select__k': 'all'}
Melhor score (neg MSE): -4288357609.3239694


In [19]:
# 7) Avaliação no conjunto de teste
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

best = gscv.best_estimator_
y_pred = best.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
err_percent = (np.mean(np.abs(y_test - y_pred) / (y_test + 1e-9))) * 100

print(f'MSE: {mse:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'R2: {r2:.4f}')
print(f'Erro Percentual Médio: {err_percent:.2f}%')

MSE: 4279352175.39
RMSE: 65416.76
MAE: 45257.53
R2: 0.8693
Erro Percentual Médio: 11.05%


In [20]:
# 8) Precisão dentro de 20% e resumo final
within_20 = ((np.abs(y_test - y_pred) / (y_test + 1e-9)) <= 0.20).mean() * 100
print(f'Precisão dentro de 20%: {within_20:.2f}%')

if rmse < 130000:
    print('\n OBJETIVO ATINGIDO: RMSE abaixo de 130.000')
else:
    print('\n OBJETIVO NÃO ATINGIDO: RMSE ainda acima de 130.000')

Precisão dentro de 20%: 86.31%

 OBJETIVO ATINGIDO: RMSE abaixo de 130.000


Conclusão

Após a aplicação das técnicas de otimização propostas, o modelo apresentou melhoria significativa em suas métricas de desempenho.
O RMSE final ficou abaixo da meta de 130.000, indicando que o modelo conseguiu realizar previsões com erro médio dentro do limite definido para o projeto.

Além disso, o coeficiente de determinação (R²) apresentou valor satisfatório, demonstrando que uma parcela expressiva da variação do preço dos imóveis pôde ser explicada pelas variáveis independentes selecionadas.

De forma geral, as etapas de limpeza dos dados, remoção de outliers e padronização, aliadas à geração de features polinomiais e à seleção automática de variáveis, foram essenciais para o ganho de desempenho obtido.

📈 O resultado final comprova que é possível, por meio de técnicas estatísticas e de engenharia de atributos, aperfeiçoar um modelo de regressão linear tradicional para atingir níveis de precisão compatíveis com o objetivo do projeto.